In [26]:
import numpy as np
import pandas as pd

from collections import defaultdict # 설정되지 않은 키에 대한 기본값 제공, 사전작업에 유용. 오류 방지
from itertools import chain, combinations  
from optparse import OptionParser #명령줄 옵션 및 인수 구문 분석

In [27]:
df = pd.read_csv('C:\\Users\\gureh\\train.csv')
df.head()

,label,achiote paste,achiote powder,acini di pepe,acorn squash,active dry yeast,adobo sauce,adobo seasoning,adzuki beans,agave nectar,...,yellow rock sugar,yellow squash,yellow summer squash,yellow tomato,yellowfin tuna,yogurt cheese,yucca root,yukon gold potatoes,yuzu juice,zucchini blossoms
0,Infineon Raceway Baked Beans,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Sour Cream Noodle Bake,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Middle-Eastern Eggplant Rounds,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Saffron Jewel Rice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Thai Sweet and Sour Wings,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
features = df.iloc[:, 1:] #dataframe 행 열 엑세스 // 첫번째 제외 모든 열 포함
labels = df.iloc[:, 0] # 모든행과 첫번째 열
#features와 labels 독립적으로 작업 가능하게 함. f=첫번째 열 제외 모든열 / ㅣ=첫째열 값 포함

In [29]:
def powerset(s): #집합을 s로 입력받고 거듭제곱 집합을 반환
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s))) #생성기 표현식 사용 s 1에서~ 까지 모든 조합 생성
            #조합을 단일 반복 가능으로 평면화

def getAboveMinSup(itemSet, itemSetList, minSup, globalItemSetWithSup): # 함수정의
    freqItemSet = set() # 빈 집합 초기화
    localItemSetWithSup = defaultdict(int) # 기본값이 0으로 설정된 defaultdict 초기화. 각 항목 지원횟수 저장

    for item in itemSet:
        for itemSet in itemSetList: 
            if item.issubset(itemSet): # item 하위 집합 확인 
                globalItemSetWithSup[item] += 1 # 지원횟수 증가
                localItemSetWithSup[item] += 1 # 

    for item, supCount in localItemSetWithSup.items():
        support = float(supCount / len(itemSetList)) # 지원횟수를 총 항목집합수로 나누어 현재 항목에 대한 지원값 계산
        if(support >= minSup): #최소보다 큰지 확인
            freqItemSet.add(item) # 크다면 추가

    return freqItemSet # 빈발항목 집합의 집합 반환


def getUnion(itemSet, length): # 
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def pruning(candidateSet, prevFreqSet, length): 
    tempCandidateSet = candidateSet.copy() # 원본 집합을 수정하지 않게 복사본 만듬
    for item in candidateSet:
        subsets = combinations(item, length) #모든 조합 생성 combination 기능 사용
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if(frozenset(subset) not in prevFreqSet):
                tempCandidateSet.remove(item) #하위 집합 중 하나 찾을 수 없어 item 제거
                break #항목 중복 제거 방지
    return tempCandidateSet 


def associationRule(freqItemSet, recipes, labels, itemSetWithSup, minConf): #(~)를 입력으로 받는 함수 정의
    rules = []
    lbls = []
    
    sets_compare = [set(re) for re in recipes] # 리스트 내 각 항목 집합으로 변환 후 변수 저장 / 연관규칙과 관련된 레이블 판단에 사용
    
    for k, itemSet in freqItemSet.items(): #사전의 각 키-값 반복 // k는 현재키 itemset에는 현재 값 할당
        for item in itemSet: 
            subsets = powerset(item) # item의 모든 부분집합 생성
            for s in subsets: # s반복
                
                ss = set([es for es in s]) # s의 모든 요소를 집합으로 변환 후 ss에 반복
                if(len(ss.intersection(sets_compare[0])) > 0 and len(ss.intersection(sets_compare[1])) > 0): # ss와 sets_compare의 첫번째 두번째 요소간 교집합 길이가 0보다 큰지 확인 // 참이면 lbls 리스트에 [labels[0], labels[1]]을 추가
                    lbls.append([labels[0], labels[1]])
                if(len(ss.intersection(sets_compare[0])) > 0): # 현재 ss와 sets_compare의 첫 번째 요소 간의 교집합의 길이가 0보다 큰지 확인합니다. 이 조건이 참이면 lbls 리스트에 labels[0]을 추가합니다.
                    lbls.append(labels[0])
                if(len(ss.intersection(sets_compare[1])) > 0): # 현재 ss와 sets_compare의 두 번째 요소 간의 교집합의 길이가 0보다 큰지 확인합니다. 이 조건이 참이면 lbls 리스트에 labels[1]을 추가합니다.
                    lbls.append(labels[1])
                    
                confidence = float( 
                    itemSetWithSup[item] / itemSetWithSup[frozenset(s)]) #현재 item의 지지도를 현재 s의 지지도로 나누어 신뢰도를 계산
                if(confidence > minConf): # 현재 신뢰도가 최소 신뢰도(minConf)보다 큰지 확인
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules, lbls


def getItemSetFromList(itemSetList): # itemSetList를 입력으로 받아들이는 getItemSetFromList 함수를 정의
    tempItemSet = set() # 빈 집합인 tempItemSet을 초기화합니다. 이 집합은 임시로 사용되며, 나중에 반환

    for itemSet in itemSetList: # itemSetList의 각 itemSet에 대해 반복 // itemSet은 리스트의 각 요소(아이템 집합)
        for item in itemSet: # 현재 itemSet에서 각 item에 대해 반복
            tempItemSet.add(frozenset([item])) # item을 하나의 요소로 가지는 불변 집합(frozenset)을 생성하고, 이를 tempItemSet에 추가

    return tempItemSet # 생성된 tempItemSet을 반환

In [30]:
def apriori(itemSetList, labels, minSup, minConf): # itemSetList, labels, minSup, minConf를 입력으로 받아들이는 apriori 함수를 정의
    C1ItemSet = getItemSetFromList(itemSetList) # itemSetList에서 1-항목 집합(C1ItemSet)을 생성
    #최종 결과 전역 빈도 항목 세트
    globalFreqItemSet = dict() # 빈 사전인 globalFreqItemSet을 초기화
    # 지원 개수로 설정된 글로벌 항목 저장
    globalItemSetWithSup = defaultdict(int) # 빈 디폴트 사전(defaultdict)인 globalItemSetWithSup을 초기화 // 전역 아이템 집합과 해당 지지도 개수를 저장하는 데 사용

    L1ItemSet = getAboveMinSup(
        C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
            #tAboveMinSup 함수를 사용하여 최소 지지도 이상인 1-항목 집합(C1ItemSet)찾기/ L1ItemSet에 할당 
        # 동시에 globalItemSetWithSup도 업데이트
    currentLSet = L1ItemSet
    k = 2 #  변수 k를 2로 초기화 // L-항목 집합이 몇 항목인지

    # 빈도 항목 집합 계산
    while(currentLSet):
        # 빈번한 항목 집합 저장
        globalFreqItemSet[k-1] = currentLSet
        # 결합 Lk
        candidateSet = getUnion(currentLSet, k)
        # 부분 집합 검정 수행 및 제거된 수퍼 세트 제거
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        # 검색 항목지원 카운트를 위한 세트
        currentLSet = getAboveMinSup(
            candidateSet, itemSetList, minSup, globalItemSetWithSup)
        k += 1

    rules, lbls = associationRule(globalFreqItemSet, itemSetList, labels, globalItemSetWithSup, minConf)
    #rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules, lbls

In [31]:
idx_to_column = {k:v for k,v in enumerate(features.columns.values)} # 열의 인덱스를 키로 / 열 이름을 values로 사용 // 사전 생성
column_to_idx = {v:k for k,v in enumerate(features.columns.values)} # 열의 인덱스를 values로 / 열의 이름을 키로 사용 // 사전 생성

In [32]:
def get_random_recipes(features, labels, nr):
    ids = np.random.randint(0, len(features), size=nr).tolist() # features의 길이 범위 내에서 nr개의 난수를 생성하여 리스트 형태로 ids에 할당
    idxs = np.array([np.arange(len(features.columns.values)).tolist() for x in range(nr)]) # features의 열 수에 해당하는 인덱스들을 생성하여 2차원 배열 idxs에 할당 // nr개의 행이 있고, 각 행은 features의 열 인덱스
    multiply = features.iloc[ids].values.astype(np.int32) * idxs # ids를 사용하여 features에서 해당하는 레시피들을 선택한 후, idxs와 곱셈 연산을 수행
    idx_arr  = [np.where(mult != 0)[0].tolist() for mult in multiply] # multiply의 각 원소를 순회하면서 0이 아닌 원소의 인덱스를 찾기 // idx_arr에 리스트 형태로 저장 // idx_arr은 선택된 레시피들에 대해 0이 아닌 원소의 열 인덱스들을 나타내는 리스트들의 리스트
    recipes = [] # 빈 리스트인 recipes를 초기화 // 선택된 레시피들에 대한 결과를 저장
    for arr_idx in idx_arr: # rr_idx는 선택된 레시피에 대한 0이 아닌 원소의 열 인덱스
        temp_result = [] #  현재 레시피에 대한 결과를 임시로 저장
        for idx in arr_idx: # arr_idx의 각 열 인덱스에 대해 반복
            temp_result.append(idx_to_column[idx]) # 사전을 사용하여 열 인덱스(idx)에 해당하는 열 이름을 찾아 temp_result에 추가
        recipes.append(temp_result)
    return recipes, labels[ids].values # 무작위로 선택된 레시피와 해당하는 레이블이 반환

In [33]:
def print_recipe(recipe, label):
    print("-" * 50)
    print("Recipe: ", label)
    print("-" * 50)
    for item in recipe:
        print(item)

In [34]:
# get 2 random receipes
recipes, lbls = get_random_recipes(features, labels, nr=2)

# print recipes
pr = [print_recipe(rec, lbl) for rec, lbl in list(zip(recipes,lbls))]

--------------------------------------------------
Recipe:  Salmon Burgers with Caesar Slaw
--------------------------------------------------
bread crumbs
egg whites
extra-virgin olive oil
parsley leaves
romaine lettuce
--------------------------------------------------
Recipe:  Spiced Chocolate Budino
--------------------------------------------------
ground cinnamon
red pepper flakes


In [35]:
freqItemSet, rules, lbls = apriori(recipes, lbls, minSup=0.5, minConf=0.5) # 레시피 데이터 / 라벨 / 최소빈도 결정 / 최소신뢰도 

In [36]:
def find_recipe(ingredients, rules, lbls):
    rules_dic = {} #  딕셔너리 rules_dic를 생성 / 규칙과 해당하는 레이블을 저장
    for i, item in enumerate(np.array(rules)[:, :-1]): # 리스트를 넘파이로 변환 마지막 열을 제외한 모든 열을 선택
        item = [list(i) for i in item] # 규칙의 항목을 리스트 형태로 변환
        rules_dic[str(item)] = lbls[i] # 셔너리에 규칙의 항목을 키로, 해당하는 레이블을 값으로 저장
    
    result = None # result 변수를 None으로 초기화
    
    try: # 예외처리
        result = rules_dic[str(ingredients)] # ingredients와 일치하는 레시피를 찾아서 result에 할당
    except:
        result = "Recipe not found"
        
    return result

In [37]:
print('Try to find recipe using ingredients: {0} and {1}'.format(recipes[0][0], recipes[0][1]))
#주어진 재료(ingredients)를 사용하여 레시피를 찾으려고 시도하는 메시지를 출력

found_recipe = find_recipe([[recipes[0][0]], [recipes[0][1]]], rules, lbls)
#find_recipe 함수를 호출하여 주어진 재료로 레시피를 찾기
print('Found recipe: ', found_recipe)

Try to find recipe using ingredients: bread crumbs and egg whites
Found recipe:  Salmon Burgers with Caesar Slaw


In [38]:
ingredients_to_try = [['vanilla vodka'], ['coconut extract']] # 테스트용으로 찾지 못하는 재료를 설정
print('Testing not finding ingredients') # 재료를 찾지 못하는 상황을 테스트 중임을 나타내는 메시지를 출력
print('Using the following ingredients: {0} and {1}'.format(ingredients_to_try[0][0], ingredients_to_try[1][0]))
print(find_recipe(ingredients_to_try, rules, lbls))

Testing not finding ingredients
Using the following ingredients: vanilla vodka and coconut extract
Recipe not found
